# Chapter 6 : Tools for Learning

## SGD 
$W = W - n * dL/dw$

$dL / dW  손실함수의 기울기$

In [1]:
class SGD:
    def __init__(self, lr = 0.01):
        self.lf = lr

    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key]


SGD의 단점

$f(x,y) = 1/20 * x^2 + y^2$ 을 생각해 보았을때 이는 종이를 양쪽 끝에서 잡아 처진 모양처럼 그려진다.

이 함수에서 기울기는 (0, 0)에서 최소값이지만, 기울기의 대부분은 (x, 0)에 가깝게 향하고 있어 지그재그로 여러번 학습해야함

지그재그로 심하게 굽어진 움직임을 보여주며, 비효율적으로 움직임을 확인할 수 있음

SGD의 단점은 비등방성함수에서 탐색경로가 매우 비효율적임

## 모멘텀 (Mometum) 

운동량을 뜻하는 단어로 물리와 관계가 있음.

$v = av - n * dL / dW$

$W = W + v

av항은 물체가 아무런 힘을 받지 않을 때 서서히 하강시키는 역할을 한다. 

지면의 마찰이나 공기저항에 해당됨.

In [2]:
import numpy as np
class Momentum:
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None
    
    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val)
        
        for key in params.keys():
            self.v[key] = self.momentum * self.v[key] - self.lr * grads[key]
            params[key] += self.v[key]

## AdaGrad

학습률의 값에 따라 학습이 굉장히 달라지기 떄문에, 학습률을 정하는 기술로 학습률감소가 있다.

처음에 크게 학습하다가 조금씩 작게 학습한다는 이야기로 실제 신경망 학습에서 자주 사용된다.

학습률을 일괄적으로 낮추는 방법에서 더욱 발전시킨것이 AdaGrad이다.

